In [6]:
# library imports
import pandas as pd

# nltk
import nltk

from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from collections import Counter
import os

# Bestbuy specifics
import bby
import bby.util as ut

from bby.util import clean_doc, get_wordnet_pos, clean_doc

import matplotlib.pyplot as plt 


In [8]:
from bby.util import load_doc
inputfilename = '../data/complaints/complaints.csv'
input_df = pd.read_csv(inputfilename)

In [9]:
input_df.columns

Index(['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue',
       'Consumer complaint narrative', 'Company public response', 'Company',
       'State', 'ZIP code', 'Tags', 'Consumer consent provided?',
       'Submitted via', 'Date sent to company', 'Company response to consumer',
       'Timely response?', 'Consumer disputed?', 'Complaint ID'],
      dtype='object')

In [13]:
keep_columns = ['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue',
        'Consumer complaint narrative', 'Company public response', 'Company',
        'Company response to consumer','Complaint ID']

new_df = input_df[keep_columns]
new_df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,Company response to consumer,Complaint ID
0,11/05/21,Checking or savings account,Checking account,Opening an account,Account opened as a result of fraud,"On XX/XX/XXXX, I received a letter in the mail...",Company has responded to the consumer and the ...,"CITIBANK, N.A.",Closed with explanation,4879994
1,06/24/19,Mortgage,VA mortgage,Trouble during payment process,None,"Dear CFPB, Please find my Complaint against Pe...",None,"PENNYMAC LOAN SERVICES, LLC.",Closed with explanation,3284591
2,12/18/15,Consumer Loan,Vehicle loan,Managing the loan or lease,None,XXXX XXXX Santander Consumer issued the follow...,None,Santander Consumer USA Holdings Inc.,Closed with explanation,1702792
3,09/30/15,Credit card,None,APR or interest rate,None,Made a purchase on my Best Buy credit card ( C...,Company chooses not to provide a public response,"CITIBANK, N.A.",Closed with explanation,1586667
4,11/14/16,Credit card,None,Payoff process,None,I paid off Best Buy Credit Card and was awaiti...,Company has responded to the consumer and the ...,"CITIBANK, N.A.",Closed with monetary relief,2206686


In [21]:
bbymask = new_df['Company'] == "Best Buy" 
bbymask2 = new_df['Company'] == "best buy"
bby_complaints = new_df[bbymask].copy()
bby_complaints2 = new_df[bbymask2].copy()
bby_complaints2.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,Company response to consumer,Complaint ID


In [22]:
new_df['Company'].unique()

array(['CITIBANK, N.A.', 'PENNYMAC LOAN SERVICES, LLC.',
       'Santander Consumer USA Holdings Inc.',
       'CITIZENS FINANCIAL GROUP, INC.',
       'TRANSUNION INTERMEDIATE HOLDINGS, INC.', 'Coinbase, Inc.',
       'HYUNDAI CAPITAL AMERICA', 'Navient Solutions, LLC.',
       'EQUIFAX, INC.', 'NATIONSTAR MORTGAGE',
       'SELECT PORTFOLIO SERVICING, INC.',
       'Mortgage Research Center, LLC', 'LoanCare, LLC',
       'BANK OF AMERICA, NATIONAL ASSOCIATION', 'GreenSky, LLC',
       'JPMORGAN CHASE & CO.', 'Resurgent Capital Services L.P.',
       'Lending Club Corp', 'PNC Bank N.A.', 'Schachter Portnoy, L.L.C.',
       'Experian Information Solutions Inc.', 'Auto Trakk, LLC',
       'WELLS FARGO & COMPANY', 'Paypal Holdings, Inc',
       'HABITAT FOR HUMANITY INTERNATIONAL, INC.', 'SYNCHRONY FINANCIAL',
       'ALLY FINANCIAL INC.', 'GOLDMAN SACHS BANK USA',
       'Google Compare Credit Cards Inc.', 'BBVA FINANCIAL CORPORATION',
       'BMW Financial Services NA, LLC',
       'Co